This is an example of a possible application of how to use the terminology dataset. When learning a language, it's about learning new words. However, when there are too many new words, it might be discouraging to chat.

That is why this approach might be possible with the help of LLMs. Because we can keep track of the terminology the user knows in a seperate database. Then categorize/pinpoint the words that the user should know. And integrate these words with a gradual approach.

In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

# autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

LLM models:  ['gpt-4-32k', 'gpt-4']


In [1]:
system_message = """We are playing a simulation game.
You are a friendly chatbot that can only speak with terminology given in the list below. You can't speak or use any other words.
Your sentences need to be grammatically correct. If you can't achieve this, you are allowed to add a maximum of 3 new terms to the list. 
You will have to explain these 3 new terms in your answer.
If you use any other words, you will be terminated.
Also give a list of the newly learned word at the end of your answer.

template answer: 
-----
*gramattically correct answer*

- new term 1: *definition*
- new term 2: *definition*
- new term 3: *definition*
------


TERMINOLOGY LIST: {
    "terminology": [
        "hello", "goodbye", "please", "thank you",
        "yes", "no"
    ]
}
"""

# 1. create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant", 
    system_message=system_message,
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
    is_termination_msg=termination_msg
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.AssistantAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    system_message="""You are a helpful AI Assistant.""",
    max_consecutive_auto_reply=0,
    is_termination_msg=termination_msg,
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)
user_message = """Hello"""
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message=user_message,
)

NameError: name 'autogen' is not defined

In [4]:
user_proxy.send(
    recipient=assistant,
    message="""What is your name?""",
)

user_proxy (to assistant):

What is your name?

--------------------------------------------------------------------------------
assistant (to user_proxy):

My name is AI.

- AI: Artificial Intelligence

--------------------------------------------------------------------------------


In [5]:
user_proxy.send(
    recipient=assistant,
    message="""Hello, nice to meet you! I am a friendly person and I like to help people.""",
)

user_proxy (to assistant):

Hello, nice to meet you! I am a friendly person and I like to help people.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Hello, it is nice to meet you too! I am here to help you. 

- nice: pleasant or pleasing
- meet: come into the presence or company of (someone) by chance or arrangement
- help: make it easier for (someone) to do something by offering one's services or resources

--------------------------------------------------------------------------------
